<a href="https://colab.research.google.com/github/Suffragette/github-slideshow/blob/master/FINAL_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


In [17]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]


In [18]:
!pip install pyspark==3.1.2
!pip install spark-nlp==3.3.1


In [19]:
import sparknlp
spark = sparknlp.start()


In [20]:
!wget -O train_data.csv https://raw.githubusercontent.com/sebischair/Medical-Abstracts-TC-Corpus/main/medical_tc_train.csv
!wget -O test_data.csv https://raw.githubusercontent.com/sebischair/Medical-Abstracts-TC-Corpus/main/medical_tc_test.csv

train_data = spark.read.csv("train_data.csv", inferSchema=True, header=True)
test_data = spark.read.csv("test_data.csv", inferSchema=True, header=True)


--2023-07-08 09:31:39--  https://raw.githubusercontent.com/sebischair/Medical-Abstracts-TC-Corpus/main/medical_tc_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14254503 (14M) [text/plain]
Saving to: ‘train_data.csv’

train_data.csv      100%[===================>]  13.59M  87.8MB/s    in 0.2s    

2023-07-08 09:31:39 (87.8 MB/s) - ‘train_data.csv’ saved [14254503/14254503]

--2023-07-08 09:31:39--  https://raw.githubusercontent.com/sebischair/Medical-Abstracts-TC-Corpus/main/medical_tc_test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting resp

In [21]:
print(train_data.columns)


['condition_label', 'medical_abstract']


In [27]:
from sparknlp.annotator import SentenceEmbeddings

# Aggregate the BERT embeddings at the sentence level
sentence_embeddings_bert = SentenceEmbeddings() \
    .setInputCols(["document", "bert_embeddings"]) \
    .setOutputCol("bert_sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")

# Define the classifier
classifier = ClassifierDLApproach() \
    .setInputCols(["document", "bert_sentence_embeddings"]) \
    .setOutputCol("class") \
    .setLabelColumn("condition_label") \
    .setMaxEpochs(10) \
    .setEnableOutputLogs(True)

# Add the classifier to the pipeline
pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    normalizer,
    stopwords_cleaner,
    word_embeddings,
    bert_embeddings,
    sentence_embeddings_bert,
    classifier
])

# Fit the pipeline to the training data
model = pipeline.fit(train_data)

# Transform the training and test data
train_data = model.transform(train_data)
test_data = model.transform(test_data)
